The subscriber continuously receives the data. For each latest record r received, apply the 3NN classification to the last 5 records before r, and compare the classification result with the Outcome label in r.

In [2]:
import paho.mqtt.client as mqtt
import pandas as pd
from io import StringIO
from sklearn.neighbors import KNeighborsClassifier

dataFrame= pd.DataFrame(columns=['Glucose','BloodPressure','Insulin','Outcome'])
knn5 = KNeighborsClassifier(n_neighbors=3)
lst_pred = []


def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe("myroom/diabetes")


def on_message(client, userdata, msg):
    global dataFrame, lst_pred, knn5
    new_record = pd.read_json(StringIO(msg.payload.decode()), typ='series')

    if len(dataFrame) > 5:
        knn5.fit(dataFrame[['Glucose','BloodPressure','Insulin']].tail(5), dataFrame['Outcome'].tail(5).astype(int))
        lst_pred = knn5.predict(new_record[['Glucose', 'BloodPressure', 'Insulin']].to_frame().T)

    print("Received myroom/diabetes: ", msg.payload)
    
    print('Prediction {}\r\n'.format(lst_pred))
    print('Actual:{}\r\n'.format(new_record['Outcome']))
    dataFrame.loc[len(dataFrame), dataFrame.columns] = new_record

    if len(dataFrame) > 1000:
        correct_predictions = sum(dataFrame['Outcome'].astype(int).tail(1000).values == lst_pred)
        print(f'Number of correct predictions in the last 1000 records: {correct_predictions}')
        mqttc.loop_stop()



        
    
# We create a client as the data subscriber and specify its actions for particular events
mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message

# Now, we connect to the data broker.
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# As a simple example, we just keep the data listening/receiving on and on...
mqttc.loop_forever()

/var/folders/yq/94jp9phd29s5g98ng2hwj0000000gn/T/ipykernel_7601/3939806494.py:40: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqttc = mqtt.Client()


Connected with result code 0


KeyboardInterrupt: 